# V. Cooper python version of A. Pendergrass script

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
# import xesmf as xe
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import copy
import pandas as pd
import cmocean as cmo
from cartopy.util import add_cyclic_point
import seaborn as sns
%matplotlib inline

import xesmf as xe

In [2]:
# File with the changes in climate: (ts, temp) (TS,T,Q)
changefile=xr.open_dataset('./demodata/changefields.nc')[['ts','gw']]
changefile3d=xr.open_dataset('./changefields.plev.nc')
# basefile3d=xr.open_dataset('./demodata/basefields.plev.nc')

###################

## Read air temperature kernel 
# ta_kernel_hybrid=ncread('kernels/t.kernel.nc','FLNT');
ta_kernel=xr.open_dataset('t.kernel.plev.nc')

## VTC add section to read in pressure levels
p_Pa=ta_kernel.plev
p_hPa=ta_kernel.lev_p 

pdiff=xr.open_dataset('dp_plev.nc').dp/100

# p=repmat(permute(repmat(p_hPa,[1 12]),[3 4 1 2]),[288 192 1 1]);
p = p_hPa.values[np.newaxis, :, np.newaxis, np.newaxis] * np.ones(ta_kernel.FLNT.shape)

In [3]:
###################

## Read in coordinate info
lat=xr.open_dataset('./kernels/PS.nc').lat
lon=xr.open_dataset('./kernels/PS.nc').lon
gw=xr.open_dataset('./kernels/t.kernel.nc').gw ## Gaussian weights for CESMgrid
# lev=ncread('kernels/t.kernel.nc','lev'); ## dont need

## Make an area weighting matrix
weight=np.tile(gw.values[:,np.newaxis], len(lon))
weight=weight/np.nansum(weight)
# print(weight.sum())

## Read surface temperature change
dts=changefile.ts

## Calculate the change in global mean surface temperature
dts_globalmean= (dts * weight).sum(dim=('lat','lon')).mean(dim='time')
print('Global mean dTS: ', dts_globalmean.values)
                 

Global mean dTS:  -0.5839361181344885


In [4]:
## Temperature feedback calculation

## Read TOA Longwave surface temperature kernel
ts_kernel=xr.open_dataset('./kernels/ts.kernel.nc').FLNT

## Multiply monthly mean TS change by the TS kernels (function of
## lat, lon, month) (units W/m2)
dLW_ts=ts_kernel *dts
#dLW_ts.mean(dim='time').plot()

## Read air temperature change [lon,lat,level,month]
dta=changefile3d.temp


In [5]:
## Read midpoint pressure for each grid cell (lat,lon,level,month), [Pa]
## VTC adjusted this above to be pressure levels
## p=ncread('p_sigma.nc','pmid')/100; %[hPa] 

In [6]:
## Crude tropopause estimate: 100 hPa in the tropics, lowering with
## cosine to 300 hPa at the poles.
x=np.cos(np.deg2rad(lat))
p_tropopause_zonalmean=300-200*x
## VTC
##p_tropopause= ...
##    repmat(permute(repmat(permute(repmat(p_tropopause_zonalmean', ...
##                                         [length(lon) 1]),[1 2 3]),[1 ...
##                    1 length(lev)]),[1 2 3 4]),[1 1 1 12]);
p_tropopause_zonalmean
p_tropopause = p_tropopause_zonalmean.values[
    np.newaxis, np.newaxis,:, np.newaxis] * np.ones(ta_kernel.FLNT.shape)
p_tropopause = xr.DataArray(p_tropopause,dims=changefile3d.dims, coords = changefile3d.coords)

In [7]:
## Set the temperature change to zero in the stratosphere (mask out stratosphere)
dta=xr.where(p>=p_tropopause, dta, np.nan)

## Convolve air temperature kernel with air temperature change
## VTC
## dLW_ta=squeeze(sum(ta_kernel.*dta,3));
dLW_ta=ta_kernel.FLNT * dta.values * pdiff.values

In [8]:

## Add the surface and air temperature response; Take the annual
## average and global area average 
dLW_t_globalmean = -(
    (dLW_ta.sum(dim='plev') + dLW_ts).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
t_feedback=dLW_t_globalmean / dts_globalmean

print('Temperature feedback: ', str(t_feedback.values), ' W m^-2 K^-1')

Temperature feedback:  -3.7459334281195757  W m^-2 K^-1


# Planck Feedback

In [9]:
## Project surface temperature change into height 
##VTC
##dts3d=repmat(permute(dts,[1 2 4 3]),[1 1 30 1]);
dts3d = dts + changefile3d.temp-changefile3d.temp

## Mask stratosphere
dt_planck = xr.where(p>=p_tropopause, dts3d, np.nan)


In [10]:
## Convolve air temperature kernel with 3-d surface air temp change
##VTC
##dLW_planck=squeeze(sum(ta_kernel_hybrid.*dt_planck,3));
# dLW_planck = squeeze(sum(ta_kernel * dt_planck.*pdiff,3))
dLW_planck=ta_kernel.FLNT * dt_planck.values * pdiff.values

## Take the annual average and global area average; incorporate the
## part due to surface temperature change itself 
dLW_planck_globalmean = -(
    (dLW_planck.sum(dim='plev') + dLW_ts).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
planck_feedback=dLW_planck_globalmean / dts_globalmean


In [11]:
## Lapse rate feedback                                                                                                                                                                 
## Calculate the departure of temperature change from the surface
## temperature change
dt_lapserate=xr.where(p>=p_tropopause, dta-dt_planck, np.nan)

## Convolve air temperature kernel with 3-d surface air temp change
## VTC
## dLW_lapserate=squeeze(sum(ta_kernel.*dt_lapserate,3));
# dLW_lapserate=squeeze(sum(ta_kernel.*dt_lapserate.*pdiff,3));
dLW_lapserate = ta_kernel.FLNT * dt_lapserate.values * pdiff.values

## Take the annual average and global area average 
dLW_lapserate_globalmean = -(
    (dLW_lapserate.sum(dim='plev')).mean(dim='time') * weight).sum()

## Divide by the global annual mean surface warming (units: W/m2/K)
lapserate_feedback = dLW_lapserate_globalmean / dts_globalmean

print('Planck feedback: ', 
      str(planck_feedback.values), ' W m^-2 K^-1')
print('Lapse rate feedback: ', 
      str(lapserate_feedback.values), ' W m^-2 K^-1')

### SANITY CHECK: Do the Planck and lapse-rate feedbacks add up to
### the total temperature feedback? (They should)

## Planck + lapse rate feedback
total_t_feedback = planck_feedback+lapserate_feedback

print('Temperature feedback: ',
      str(t_feedback.values), ' W m^-2 K^-1')
print('Planck+lapse rate components: ',
      str(total_t_feedback.values), ' W m^-2 K^-1')


Planck feedback:  -3.261498255128057  W m^-2 K^-1
Lapse rate feedback:  -0.48443517299151895  W m^-2 K^-1
Temperature feedback:  -3.7459334281195757  W m^-2 K^-1
Planck+lapse rate components:  -3.7459334281195757  W m^-2 K^-1
